In [1]:
import tensorflow as tf
import numpy as np
import pickle
import random
import spacy
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

2021-12-12 12:43:36.485457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-12 12:43:36.485483: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dataframe = pd.read_csv("../Data/dataset.csv")
print(dataframe)

     Unnamed: 0                                             inputs  \
0             0                             Oi;Eae;Olá;Bom dia;iai   
1             1           Thau;Te vejo depois;Falou;Adeus;até mais   
2             2  Valeu;Isso vai ser útil;Muito obrigado;Ok obri...   
3             3  Como você está?;Como você vai?;Tudo bem?;Vou b...   
4             4  Vou bem;Estou bem;Me sinto bem;Tô de boa;Tô be...   
..          ...                                                ...   
151         151         Você está contente;você se sente contente?   
152         152                            que bom;que ótimo;ótimo   
153         153                    você é chata;você é muito chata   
154         154  me conte uma piada;diz uma piada;diz uma piada...   
155         155  qual a sua performance?;qual é a performance a...   

                                               outputs  
0                           Olá como vai?;Oi como vai?  
1            Te vejo depois;Até breve;Até um 

In [ ]:
!python -m spacy download pt_core_news_sm 

In [3]:
nlp = spacy.load('pt_core_news_sm')
words = []
labels = []
docs_x = []
docs_y = []

for i in range(dataframe.shape[0]):
    for inp in dataframe.iloc[i]["inputs"].split(";"):
        doc = nlp(inp.lower())
        wrds = [token.lemma_+token.pos_ for token in doc if token.pos_ != "PUNCT"]

        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(i)
    if i not in labels:
        labels.append(i)

words = sorted(list(set(words)))
labels = sorted(labels)
print(len(words))
print(words[20:35])
        

495
['algumDET', 'algumPRON', 'alguémPRON', 'almoçarVERB', 'alterarVERB', 'amanhãADV', 'amarADV', 'amarVERB', 'amigoNOUN', 'amizadeNOUN', 'amoVERB', 'andarVERB', 'aniversárioNOUN', 'anoNOUN', 'aprenderVERB']


In [4]:
docs_x

[['oiVERB'],
 ['eaeADV'],
 ['oláAUX'],
 ['bomADJ', 'diaNOUN'],
 ['iaiDET'],
 ['thauADJ'],
 ['tePRON', 'verVERB', 'depoisADV'],
 ['falarVERB'],
 ['adeusPRON'],
 ['atéADP', 'maisADV'],
 ['valerVERB'],
 ['issoPRON', 'irAUX', 'serAUX', 'útilADJ'],
 ['muitoADV', 'obrigarVERB'],
 ['okAUX', 'obrigarVERB'],
 ['agradecerVERB'],
 ['comerADV', 'vocêPRON', 'estarAUX'],
 ['comerADV', 'vocêPRON', 'irVERB'],
 ['tudoPRON', 'bemADV'],
 ['irAUX', 'bemADV', 'eCCONJ', 'vocêPRON'],
 ['irAUX', 'bemADV'],
 ['estarAUX', 'bemADV'],
 ['mePRON', 'sentirVERB', 'bemADV'],
 ['tôNOUN', 'deADP', 'bomADJ'],
 ['tôADP', 'bemADV'],
 ['tôCCONJ', 'ótimoADJ'],
 ['irAUX', 'mauADJ'],
 ['nãoADV', 'estarAUX', 'bemADV'],
 ['mePRON', 'sentirVERB', 'mauADJ'],
 ['nãoADV', 'estarAUX', 'mePRON', 'sentirVERB', 'bemADV'],
 ['tôCCONJ', 'mauADJ'],
 ['doADP', 'quePRON', 'vocêPRON', 'gostarVERB'],
 ['oPRON', 'quePRON', 'vocêPRON', 'gostarVERB'],
 ['gostarVERB', 'deSCONJ', 'fazerVERB', 'algumDET', 'coisaNOUN'],
 ['gostarVERB', 'deSCONJ', 'f

In [5]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = []
    
    for token in doc:
        wrds.append(token)

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    bag = np.array(bag)
    bag = bag.reshape((1,bag.shape[0],1))

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)
N_CLASSES = len(output[0])
SHAPE = training[0].shape[1]
print(SHAPE,training[0].shape)

495 (1, 495, 1)


In [6]:
HIDDEN_NEURONS = int((training[0].shape[1]+len(output[0]))/2)

In [7]:
#entradas+saídas/2 (opcional).
def dense_layers(inputs):
    x = tf.keras.layers.Conv2D(8,3,activation='relu',input_shape=(1,SHAPE,1),padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D(padding='same')(x)

    x = tf.keras.layers.Conv2D(16,3,activation='relu',padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(padding='same')(x)
    #x = tf.keras.layers.Conv2D(16,3,activation='relu',padding='same')(x)

    x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(HIDDEN_NEURONS,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(HIDDEN_NEURONS//2,activation='relu')(x)
    
    #y = tf.keras.layers.Dense(78/2,activation='relu')(x)
    #y = tf.keras.layers.Dropout(0.2)(y)

    #concatted = tf.keras.layers.Concatenate()([x, y])

    #x = tf.keras.layers.BatchNormalization()(x)
    return x

def classfier_layer(x,N_CLASSES):
    x = tf.keras.layers.Dense(N_CLASSES,activation='softmax',name='classification')(x)
    return x

def final_model(inputs,N_CLASSES):
    dense = dense_layers(inputs)
    
    classfier = classfier_layer(dense,N_CLASSES)
    
    model = tf.keras.Model(inputs=inputs,outputs=classfier)
    
    return model
    
def define_and_compile_model(SHAPE,N_CLASSES):
    inputs = tf.keras.layers.Input(shape=(1,SHAPE,1))
    
    # create the model
    model = final_model(inputs,N_CLASSES)
    
    # compile your model
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics = {'classification' : 'accuracy'})

    return model

In [8]:
model = define_and_compile_model(SHAPE,N_CLASSES)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 495, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 1, 495, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 248, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 248, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 124, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1984)              0     

2021-12-12 12:44:12.686359: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 12:44:12.687649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-12 12:44:12.687967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-12 12:44:12.688244: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-12 12:44:12.688459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [10]:
!rm -r -f logs

In [11]:
from datetime import datetime

logdir = "logs\\fit\\train_"+ datetime.now().strftime("%Y-%m-%d:%H:%M:%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(training,output,epochs=50,callbacks=[tensorboard_callback])

Epoch 1/50
16/16 [==============================] - 2s 26ms/step - loss: 0.5506 - accuracy: 0.0041
Epoch 2/50
16/16 [==============================] - 0s 24ms/step - loss: 0.0760 - accuracy: 0.0061
Epoch 3/50
16/16 [==============================] - 0s 25ms/step - loss: 0.0640 - accuracy: 0.0041
Epoch 4/50
16/16 [==============================] - 0s 23ms/step - loss: 0.0426 - accuracy: 0.0020
Epoch 5/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0399 - accuracy: 0.0163
Epoch 6/50
16/16 [==============================] - 0s 23ms/step - loss: 0.0397 - accuracy: 0.0122
Epoch 7/50
16/16 [==============================] - 0s 23ms/step - loss: 0.0395 - accuracy: 0.0061
Epoch 8/50
16/16 [==============================] - 0s 20ms/step - loss: 0.0396 - accuracy: 0.0122
Epoch 9/50
16/16 [==============================] - 0s 16ms/step - loss: 0.0394 - accuracy: 0.0081
Epoch 10/50
16/16 [==============================] - 0s 16ms/step - loss: 0.0395 - accuracy: 0.0061
Epoch 11/

In [26]:
### tensorboard --logdir "logs\\fit"

In [12]:
def bag_of_words(s):
    bag = [0 for _ in range(len(words))]

    doc = nlp(s.lower())
    s_words = []
    for token in doc:
        if token.pos_ != "PUNCT":
            s_words.append(token.lemma_+token.pos_)

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    bag = np.array(bag)
    bag = bag.reshape(1,1,bag.shape[0],1)
    return bag

In [18]:
while True:
    inp = str(input("Digite: "))
    if inp == "quit":
        break
        
    bag = bag_of_words(inp)
    print(bag.shape)
    predictions = model.predict(bag)
    results_index = np.argmax(predictions)

    tag = labels[results_index]

    responses = dataframe.iloc[tag]['outputs'].split(";")

    response = random.choice(responses)
    confidence = predictions[0][results_index]
    print("Model:",response,confidence)

(1, 1, 495, 1)
Model: Olá como vai? 0.93050504
(1, 1, 495, 1)
Model: Olá como vai? 0.88617146


In [57]:
import json

test = None
with open("../Data/test.json","r",encoding='utf-8') as f:
    test = json.load(f)['intents']
print(test[0:5])

[{'tag': 'tag0', 'patterns': ['Eae', 'Bom dia', 'iai', 'oi minerva'], 'responses': ['Olá como vai?', 'Oi como vai?'], 'context_set': ''}, {'tag': 'tag1', 'patterns': ['Falou', 'Adeus', 'te vejo depos', 'Até breve', 'até mais'], 'responses': ['Te vejo depois', 'Até breve', 'Até um outro dia']}, {'tag': 'tag2', 'patterns': ['Muitíssimo obrigado', 'obrigado', 'Agradeço', 'agradecido'], 'responses': ['Que bom que isso te ajudou', 'De nada', 'E um prazer lhe ajudar', 'Pode sempre contar comigo', 'não há de que', 'de nada', 'estou muito animado por você']}, {'tag': 'tag3', 'patterns': ['Como você tá?', 'Tudo bem?', 'tá tudo bem?', 'como você está?'], 'responses': ['Estou bem, e você?', 'Vou bem, e você?', 'Me sinto otima,e você como se sente?', 'Eu estou muito bem obrigada por perguntar, e você?', '<status>']}, {'tag': 'tag4', 'patterns': ['Me sinto ótimo', 'Tô bem', 'estou bem', 'estou ótimo', 'Tô ótimo'], 'responses': ['Que ótimo', 'Fico feliz em saber que se sente bem', 'Que bom']}]


In [53]:
def bag_of_words_test(s):
    bag = [0 for _ in range(len(words))]

    doc = nlp(s.lower())
    s_words = []
    for token in doc:
        if token.pos_ != "PUNCT":
            s_words.append(token.lemma_+token.pos_)

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    bag = np.array(bag)
    return bag.reshape(1,bag.shape[0],1)

In [58]:
test_y = []
test_x = []

for obj in test:
    for sentence in obj["patterns"]:
        test_x.append(bag_of_words_test(sentence))
        test_y.append(int(obj["tag"].replace("tag","")))

test_x = np.array(test_x)
print(test_x.shape)

(109, 1, 504, 1)


In [59]:
predictions = model.predict(test_x)
predictions = [np.argmax(x) for x in predictions]
print(predictions[0:10])

[0, 0, 0, 117, 1, 1, 120, 1, 1, 2]


In [60]:
from sklearn.metrics import accuracy_score

print(accuracy_score(predictions,test_y))

0.8715596330275229


In [68]:
def export_data_csv(result_data):
    result_data = result_data.split("\n")
    formatted_data = ""
    for line in result_data:
        formatted_data += line.replace(" ",",")+"\n"
    with open("result_data_train.csv","w") as f:
        f.write(formatted_data)

In [78]:
from sklearn.metrics import classification_report

export_data = classification_report(predictions,test_y,output_dict=True)
export_data = pd.DataFrame(export_data)
export_data.to_excel("result_data_train.xlsx")

C:\Users\Michel\anaconda3\envs\ia\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Michel\anaconda3\envs\ia\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Michel\anaconda3\envs\ia\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
model.save("model_4.h5")

In [17]:
!ls



'logs\fit\train_2021-12-12:12:44:51'   model_4.h5	        Train.ipynb
 model_2.h5			       model.h5
 model_3.h5			       result_data_train.xlsx


In [16]:
with open("../Data/dataV2.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)